## Introduction

This is not the best solution possible, but it is a solution. One can probably implement something more robust by using libraries like [marko](https://github.com/frostming/marko) or [mistletoe](https://github.com/miyuchina/mistletoe).

This solution converts a markdown style document into a Google Apps Script code that in turn creates a Google Forms.

## Creating functions

In [ ]:
import logging
import re
from typing import Any
# from IPython.display import HTML
# from ipywidgets import HTML

In [ ]:
_logger = logging.getLogger(__name__)


def begin_create_form():
    _logger.debug('Creating form')
    return 'function createForm() {\n'


def end_create_form():
    _logger.debug('Form created')
    return '}'


def _concatenate_lines(lines: list[str] | tuple[str], identation_level: int = 1, identation: str = 2 * ' '):
    return '\n'.join([f'{identation * identation_level}{line}' for line in lines])


def create_form(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    confirmation_message = kwargs.get('confirmation_message', '')

    lines = [f'var form = FormApp.create("{title}")']

    if len(description) > 0:
        lines.append(f'  .setDescription("{description}")')

    if len(confirmation_message) > 0:
        lines.append(f'  .setConfirmationMessage("{confirmation_message}")')

    lines[-1] += ';\n'

    lines.append('var sections = {};\n')

    return _concatenate_lines(lines)


def create_section(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addpagebreakitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')

    lines = ['var section = form.addPageBreakItem()',
             f'  .setTitle("{title}");\n',
             f'sections["{title}"] = section;\n']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    _logger.debug(f'Creating section: {title} - {description}')

    return '\n' + _concatenate_lines(lines)


def edit_section(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addpagebreakitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')

    lines = [f'sections["{title}"]',
             f'  .setTitle("{title}")']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    lines[-1] += ';\n'

    _logger.debug(f'Editing section: {title} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_title_and_description_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addsectionheaderitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')

    lines = ['form.addSectionHeaderItem()',
             f'  .setTitle("{title}")']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    lines[-1] += ';\n'

    _logger.debug(f'Creating title and description item: {title} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_short_text_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addtextitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)

    lines = ['form.addTextItem()',
             f'  .setTitle("{title}")']

    if required:
        lines.append('  .setRequired(true)')

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    lines[-1] += ';\n'

    _logger.debug(f'Creating short text item: {title}{" (required)" if required else ""} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_paragraph_text_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addparagraphtextitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)

    lines = ['form.addParagraphTextItem()',
             f'  .setTitle("{title}")']

    if required:
        lines.append('  .setRequired(true)')

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    lines[-1] += ';\n'

    _logger.debug(f'Creating paragraph text item: {title}{" (required)" if required else ""} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_multiple_choice_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addmultiplechoiceitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)
    choices = kwargs.get('choices', [])

    if any(c.startswith('item.createChoice(') for c in choices):
        lines = ['var item = form.addMultipleChoiceItem()',
                 f'  .setTitle("{title}");\n']

        lines.append('item.setChoices([')
        for choice in choices:
            lines.append(f'    {choice},')

        lines[-1] = lines[-1][:-1]
        lines.append('  ])')

    else:
        lines = ['form.addMultipleChoiceItem()',
                 f'  .setTitle("{title}")',
                 f'  .setChoiceValues({choices})']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    # TODO .showOtherOption(true);

    _logger.debug(f'Creating multiple choice item: {title}{" (required)" if required else ""} - {description}\nchoices: {choices}')

    return '\n' + _concatenate_lines(lines)


def create_checkbox_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addcheckboxitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)
    choices = kwargs.get('choices', [])

    if any(c.startswith('item.createChoice(') for c in choices):
        lines = ['var item = form.addMultipleChoiceItem()',
                 f'  .setTitle("{title}");']

        lines.append('item.setChoices([')
        for choice in choices:
            lines.append(f'    {choice},')

        lines[-1] = lines[-1][:-1]
        lines.append('  ])')

    else:
        lines = ['form.addMultipleChoiceItem()',
                 f'  .setTitle("{title}")',
                 f'  .setChoiceValues({choices})']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating checkbox item: {title}{" (required)" if required else ""} - {description}\nchoices: {choices}')

    return '\n' + _concatenate_lines(lines)


def create_list_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addlistitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)
    choices = kwargs.get('choices', [])

    if any(c.startswith('item.createChoice(') for c in choices):
        lines = ['var item = form.addMultipleChoiceItem()',
                 f'  .setTitle("{title}");']

        lines.append('item.setChoices([')
        for choice in choices:
            lines.append(f'    {choice},')

        lines[-1] = lines[-1][:-1]
        lines.append('  ])')

    else:
        lines = ['form.addMultipleChoiceItem()',
                 f'  .setTitle("{title}")',
                 f'  .setChoiceValues({choices})']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating list item: {title}{" (required)" if required else ""} - {description}\nchoices: {choices}')

    return '\n' + _concatenate_lines(lines)


def create_scale_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addscaleitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)
    min_label = kwargs.get('min_label', '')
    max_label = kwargs.get('max_label', '')
    min_value = kwargs.get('min', 0)
    max_value = kwargs.get('max', -1)

    lines = ['form.addScaleItem()',
             f'  .setTitle("{title}")',
             f'  .setBounds({min_value}, {max_value})',
             f'  .setLabels("{min_label}", "{max_label}")']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating scale item: {title}{" (required)" if required else ""} - {description}\nmin ({min_label}): {min_value} - max ({max_label}): {max_value}')

    return '\n' + _concatenate_lines(lines)


def create_date_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#adddateitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)

    lines = ['form.addDateItem()',
             f'  .setTitle("{title}")']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating date item: {title}{" (required)" if required else ""} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_time_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addtimeitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)

    lines = ['form.addTimeItem()',
             f'  .setTitle("{title}")']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating time item: {title}{" (required)" if required else ""} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_date_time_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#adddatetimeitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)

    lines = ['form.addDateTimeItem()',
             f'  .setTitle("{title}")']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating date time item: {title}{" (required)" if required else ""} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_duration_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#adddurationitem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)

    lines = ['form.addDurationItem()',
             f'  .setTitle("{title}")']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating duration item: {title}{" (required)" if required else ""} - {description}')

    return '\n' + _concatenate_lines(lines)


def create_grid_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addgriditem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)
    rows = kwargs.get('rows', [])
    columns = kwargs.get('columns', [])

    lines = ['form.addGridItem()',
             f'  .setTitle("{title}")',
             f'  .setRows({rows})',
             f'  .setColumns({columns})']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating grid item: {title}{" (required)" if required else ""} - {description}\nrows: {rows}\ncolumns: {columns}')

    return '\n' + _concatenate_lines(lines)


def create_checkbox_grid_item(**kwargs) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#addcheckboxgriditem
    title = kwargs.get('title', '')
    description = kwargs.get('description', '')
    required = kwargs.get('required', False)
    rows = kwargs.get('rows', [])
    columns = kwargs.get('columns', [])

    lines = ['form.addCheckboxGridItem()',
             f'  .setTitle("{title}")',
             f'  .setRows({rows})',
             f'  .setColumns({columns})']

    if len(description) > 0:
        lines.append(f'  .setHelpText("{description}")')

    if required:
        lines.append('  .setRequired(true)')

    lines[-1] += ';\n'

    _logger.debug(f'Creating checkbox grid item: {title}{" (required)" if required else ""} - {description}\nrows: {rows}\ncolumns: {columns}')

    return '\n' + _concatenate_lines(lines)


def move_section_to_end_of_form(title: str) -> str:
    # https://developers.google.com/apps-script/reference/forms/form#moveitemto

    lines = [f'form.moveItem(form.getItemById(sections["{title}"].getId()), form.getItems().length - 1);\n']
    # lines = [f'form.moveItem(sections["{title}"], form.getItems().length - 1);\n']

    _logger.debug(f'Moving section to end of form: {title}')

    return '\n' + _concatenate_lines(lines)


def _reset_args(args: dict[str, Any]) -> None:
    args['title'] = ''
    args['description'] = ''
    args['required'] = False
    args['choices'] = []
    args['rows'] = []
    args['columns'] = []


def main(markdown_file: str) -> None:
    main_title_regex = re.compile(r'^#[\s]*(.*)$')
    confirmation_message_regex = re.compile(r'^_(.*)_$')
    section_regex = re.compile(r'^##[\s]*(.*)$')
    title_regex = re.compile(r'^###[\s]*(.*)$')
    combobox_regex = re.compile(r'^-[\s]*(.*)$')
    radio_button_regex = re.compile(r'^\*[\s]*(.*)$')
    checkbox_regex = re.compile(r'^([-*][\s]*)?\[[\s]*\] (.*)$')
    navigation_regex = re.compile(r'^(.*) \[(.*)\]$')
    paragraph_regex = re.compile(r'^[\s]*```(.|\s)*```[\s]*$')
    short_text_regex = re.compile(r'^`(.*)`$')
    required_regex = re.compile(r'^\*\*(.*)\*\*$')
    scale_regex = re.compile(r'^(.*) (\d)+ --- (\d)+ (.*)$')
    date_regex = re.compile(r'^dd|[\d]{2}/mm|[\d]{2}/yyyy|[\d]{4}$')
    time_regex = re.compile(r'^hh|[\d]{2}:mm|[\d]{2}$')
    date_time_regex = re.compile(r'^dd|[\d]{2}/mm|[\d]{2}/yyyy|[\d]{4} hh|[\d]{2}:mm|[\d]{2}$')
    duration_regex = re.compile(r'^hh|[\d]{2}:mm|[\d]{2}:ss|[\d]{2}$')
    column_row_radio_button_grid_regex = re.compile(r'^####[\s]*(.*)$')
    column_row_checkbox_grid_regex = re.compile(r'^####[\s]*\[[\s]*\] (.*)$')


    current_function = None
    grid = False
    code = begin_create_form()
    created_main_title = False
    created_first_item = False

    row = False

    args = {
        'title': '',
        'confirmation_message': '',
        'description': '',
        'required': False,
        'choices': [],
        'rows': [],
        'columns': [],
        'min': -1,
        'max': -1,
        'min_label': '',
        'max_label': '',
    }

    for i, line in enumerate(markdown_file.split('\n')):
        _logger.debug(f'Processing line {i}: {line}')
        # the order of function calls here matters
        line = line.strip()
        if len(line) == 0:
            continue

        match = column_row_checkbox_grid_regex.match(line)
        if match is not None:
            row = match.group(1).strip().lower() == 'rows'
            current_function = create_checkbox_grid_item
            grid = True
            continue

        match = column_row_radio_button_grid_regex.match(line)
        if match is not None:
            row = match.group(1).strip().lower() == 'rows'
            current_function = create_grid_item
            grid = True
            continue

        # when reaching a new title or section, create the previous item
        match = title_regex.match(line)
        if match is not None:
            if current_function is not None:
                if current_function == create_form:
                    created_main_title = True

                code += current_function(**args)
                grid = False

                if current_function == edit_section:
                    code += move_section_to_end_of_form(args['title'])

                _reset_args(args)
                current_function = None

            elif created_first_item:
                code += create_title_and_description_item(**args)
                grid = False
                _reset_args(args)

            else:
                created_first_item = True

            line = match.group(1)
            match = required_regex.match(line)
            if match is not None:
                args['required'] = True
                args['title'] = match.group(1)

            else:
                args['title'] = line
            continue

        match = section_regex.match(line)
        if match is not None:
            if current_function is not None:
                if current_function == create_form:
                    created_main_title = True

                code += current_function(**args)
                grid = False
                _reset_args(args)
                current_function = None

            elif created_first_item:
                code += create_title_and_description_item(**args)
                grid = False
                _reset_args(args)

            else:
                created_first_item = True

            args['title'] = match.group(1)
            current_function = edit_section
            continue

        match = main_title_regex.match(line)
        if match is not None:
            if created_main_title:
                raise Exception('Main title already created')

            args['title'] = match.group(1)
            current_function = create_form
            continue

        match = confirmation_message_regex.match(line)
        if match is not None:
            confirmation_message = match.group(1)
            args['confirmation_message'] = confirmation_message
            continue

        match = paragraph_regex.match(line)
        if match is not None:
            current_function = create_paragraph_text_item
            continue

        match = short_text_regex.match(line)
        if match is not None:
            current_function = create_short_text_item
            continue

        match = checkbox_regex.match(line)
        if match is not None:
            if created_first_item:
                if grid:
                    if row:
                        args['rows'].append(match.group(2))
                    else:
                        args['columns'].append(match.group(2))

                else:
                    option = match.group(2)
                    match = navigation_regex.match(option)
                    if match is not None:
                        option = match.group(1)
                        section = match.group(2)
                        args['choices'].append(f'item.createChoice("{option}", sections["{section}"])')

                    else:
                        args['choices'].append(option)

                    current_function = create_checkbox_item

            else:
                if current_function == create_form:
                    created_main_title = True
                    code += current_function(**args)
                    _reset_args(args)
                    current_function = None
                    grid = False

                code += create_section(title=match.group(2))

            continue

        match = radio_button_regex.match(line)
        if match is not None:
            if created_first_item:
                if grid:
                    if row:
                        args['rows'].append(match.group(1))
                    else:
                        args['columns'].append(match.group(1))

                else:
                    option = match.group(1)
                    match = navigation_regex.match(option)
                    if match is not None:
                        option = match.group(1)
                        section = match.group(2)
                        args['choices'].append(f'item.createChoice("{option}", sections["{section}"])')

                    else:
                        args['choices'].append(option)

                    current_function = create_multiple_choice_item

            else:
                if current_function == create_form:
                    created_main_title = True
                    code += current_function(**args)
                    _reset_args(args)
                    current_function = None
                    grid = False

                code += create_section(title=match.group(1))

            continue

        match = combobox_regex.match(line)
        if match is not None:
            if created_first_item:
                if grid:
                    if row:
                        args['rows'].append(match.group(1))
                    else:
                        args['columns'].append(match.group(1))

                else:
                    option = match.group(1)
                    match = navigation_regex.match(option)
                    if match is not None:
                        option = match.group(1)
                        section = match.group(2)
                        args['choices'].append(f'item.createChoice("{option}", sections["{section}"])')

                    else:
                        args['choices'].append(option)
                    current_function = create_list_item

            else:
                if current_function == create_form:
                    created_main_title = True
                    code += current_function(**args)
                    _reset_args(args)
                    current_function = None
                    grid = False

                code += create_section(title=match.group(1))

            continue

        match = scale_regex.match(line)
        if match is not None:
            args['min_label'] = match.group(1)
            args['min'] = match.group(2)
            args['max'] = match.group(3)
            args['max_label'] = match.group(4)
            current_function = create_scale_item
            continue

        match = date_time_regex.match(line)
        if match is not None:
            current_function = create_date_time_item
            continue

        match = date_regex.match(line)
        if match is not None:
            current_function = create_date_item
            continue

        match = duration_regex.match(line)
        if match is not None:
            current_function = create_duration_item
            continue

        match = time_regex.match(line)
        if match is not None:
            current_function = create_time_item
            continue

        args['description'] = line

    # finished reading the file, create the last item
    if current_function is not None:
        code += current_function(**args)
        _reset_args(args)
        current_function = None

    code += end_create_form()

    return code

## Creating markdown

In [ ]:
markdown_file = """
# Forms Title

_Thanks for testing this script!_

This is a test script to convert a markdown file to a Google Forms script. It is still in development, so it may not work as expected. If you want to have more than one section, you must first explicit their names here:

- Section 2
- Section 3

### **Radio buttons**

Use `*` at the start of each line to indicate that the list is a radio button list. If you want any item to be required, write the text between `**`.

* Radio Navigation Option 1 [Section 2]
* Radio Navigation Option 2 [Section 3]

### Title and description

Use this to add a title and a description to the form.



## Section 2

Section 2 description

### Short text 1

`This indicates a short text field, and its contents will be ignored.`

### Long text 1

```
This indicates a long text field, and its contents will be ignored.
```

### Title and description 1

Use this to add a title and a description to the form.

### **Radio buttons 1**

Use `*` at the start of each line to indicate that the list is a radio button list. If you want any item to be required, write the text between `**`.

* Radio Option 1
* Radio Option 2
* Radio Option 3

### Combobox 1

Use `-` at the start of each line to indicate that the list is a radio button list.

- Combo Option 1
- Combo Option 2
- Combo Option 3

### Checkboxes 1

Use `[]` at the start of each line to indicate that the list is a checkbox list. It doesn't matter if it starts with `-` or `*`, or even if the checkoxes have a space or not inside.

- [ ] Check Option 1
[ ] Check Option 2
[] Check Option 3

### Scale 1

To create a scale, simply write the lowest label and its value, then a `---`, and finally the highest value and its label, in this order.

Lowest 1 --- 5 Highest

### Date 1

To create a date item, simply write this date format.

dd/mm/yyyy

### Time 1

To create a time item, simply write this time format.

hh:mm

### Date and time 1

To create a date and time item, simply write this date and time format.

dd/mm/yyyy hh:mm

### Duration 1

To create a duration item, simply write this duration format.

hh:mm:ss

### RadioButton Grid

To create a radio button grid, simply write the rows and columns. Note that the rows **MUST BE** written as `#### rows` and the columns **MUST BE** written as `#### columns`, ignoring case.

#### Rows

- Row 1
- Row 2
- Row 3

#### Columns

- Column 1
- Column 2
- Column 3

### Checkbox Grid

To create a checkbox grid, simply write the rows and columns. Note that the rows **MUST BE** written as `#### [] rows` and the columns **MUST BE** written as `#### [] columns`, ignoring case.

#### [] Rows

- Row 1
- Row 2
- Row 3

#### [] Columns

- Column 1
- Column 2
- Column 3



## Section 3

Section 3 description

### Short text 2

`This indicates a short text field, and its contents will be ignored.`

### Long text 2

```
This indicates a long text field, and its contents will be ignored.
```

### Title and description 2

Use this to add a title and a description to the form.

### **Radio buttons 2**

Use `*` at the start of each line to indicate that the list is a radio button list. If you want any item to be required, write the text between `**`.

* Radio Option 1
* Radio Option 2
* Radio Option 3

### Combobox 2

Use `-` at the start of each line to indicate that the list is a radio button list.

- Combo Option 1
- Combo Option 2
- Combo Option 3

### Checkboxes 2

Use `[]` at the start of each line to indicate that the list is a checkbox list. It doesn't matter if it starts with `-` or `*`, or even if the checkoxes have a space or not inside.

- [ ] Check Option 1
[ ] Check Option 2
[] Check Option 3

### Scale 2

To create a scale, simply write the lowest label and its value, then a `---`, and finally the highest value and its label, in this order.

Lowest 1 --- 5 Highest

### Date 2

To create a date item, simply write this date format.

dd/mm/yyyy

### Time 2

To create a time item, simply write this time format.

hh:mm

### Date and time 2

To create a date and time item, simply write this date and time format.

dd/mm/yyyy hh:mm

### Duration 2

To create a duration item, simply write this duration format.

hh:mm:ss

### RadioButton Grid

To create a radio button grid, simply write the rows and columns. Note that the rows **MUST BE** written as `#### rows` and the columns **MUST BE** written as `#### columns`, ignoring case.

#### Rows

- Row 1
- Row 2
- Row 3

#### Columns

- Column 1
- Column 2
- Column 3

### Checkbox Grid

To create a checkbox grid, simply write the rows and columns. Note that the rows **MUST BE** written as `#### [] rows` and the columns **MUST BE** written as `#### [] columns`, ignoring case.

#### [] Rows

- Row 1
- Row 2
- Row 3

#### [] Columns

- Column 1
- Column 2
- Column 3

"""

## Converting to Google Apps Script

In [ ]:
javascript_code = main(markdown_file)
print(javascript_code)
# print()
# HTML(data=f"<button onclick=navigator.clipboard.writeText('{javascript_code}')>Copy code</button>")

Then, paste the generated code on a [new project](https://script.google.com/home/projects/create) in Google Apps Script and execute it. On the first run of this new project it will ask for permissions to your Google Drive, which should be conceded so it can create the new form. A new file will be created on your [Google Drive](https://drive.google.com/) with the name you used as title. Note that the form is not ready to use, but at least the basic structure will be done